Code for scraping from indeed.co.uk

In [7]:
#import packages

import requests, bs4, time
import pandas as pd
import os
import scraping_funcs_indeeduk as sf


In [13]:
#scraping code:

#decide what search term to use for finding jobs
searchTerm="data"

#create empty data frame with column headers
ads=pd.DataFrame(columns=['company','job_title','salary','location','description','date','full_description','other_deets'])

# loop for scraping

for i in range(0,1000,10):
    time.sleep(1) #ensuring at least 1 second between page grabs
    url = "https://www.indeed.co.uk/jobs?q="+searchTerm+"&filter=0&l="+"&start="+str(i)
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.content)
    df=pd.DataFrame()
    df['company'] = sf.extract_company_from_result(soup)
    df['job_title'] = sf.extract_job_title_from_result(soup)
    df['salary'] = sf.extract_salary_from_result(soup)
    df['location'] = sf.extract_location_from_result(soup)
    df['description'] = sf.extract_description_from_result(soup)
    df['date'] = sf.extract_date_from_result(soup)
    
    sub_urls=extract_links(soup)
    text_list=[]
    deets_list=[]
    for j in sub_urls:
        res_sub = requests.get(j)
        soup_sub = bs4.BeautifulSoup(res_sub.content)
        desc=sf.extract_full_desc(soup_sub)
        text_list.append(desc)
        other_deets=sf.extract_headlines_from_result(soup_sub)
        deets_list.append(other_deets)
    df['full_description']=text_list
    df['other_deets']=deets_list
    ads=ads.append(df)
    

In [14]:
#establish how many ads had no salary
ads[ads['salary']=="Nothing_found"].shape

(655, 8)

In [27]:
ads['description']=[x.lstrip('\n').rstrip('\n') for x in ads['description']]
ads['company']=[x.lstrip('\n\n')for x in ads['company']]
ads['full_description']=[str(x).lstrip("['").rstrip("]'") for x in ads['full_description']]
ads['other_deets']=[str(x).lstrip("['").rstrip("]'") for x in ads['other_deets']]
ads['salary']=[x.lstrip('\n\n')for x in ads['salary']]
ads.head()

,company,job_title,salary,location,description,date,full_description,other_deets
0,Civil Service Fast Stream,Fast Stream - Government Social Research Scheme,"£27,000 a year",Nothing_found,"Your work might involve developing surveys, a...",Nothing_found,The award-winning Fast Stream is the leadershi...,"United Kingdom_Full-time, Permanent_£27,000 a ..."
1,BP,Quantitative Trader Programme,Nothing_found,Nothing_found,If your knowledge stretches even further to m...,Nothing_found,"""Job profile summary:\nResponsible for support...",London
2,Micro Focus,Data Solutions Architect,Nothing_found,Nothing_found,"Directs, manages, leads, and coordinates the ...",Nothing_found,Data Solutions Architect\n\nJob Description:\n...,Newbury RG14
3,Tangoe Inc.,Client Data Analyst,Nothing_found,Nothing_found,You’ll be responsible for monitoring volumes ...,Nothing_found,Client Data Analyst What you’ll do: This role ...,Abingdon
4,Foreign & Commonwealth Office,Open Source Intelligence Analyst,"£34,053 a year",Nothing_found,"By the time you leave us, you will have an en...",Nothing_found,The Foreign & Commonwealth Office (FCO) is at ...,"London_Full-time, Contract_£34,053 a year"


In [16]:
#print data to csv
#ads.to_csv(r"indeed_scrape_data.csv")

In [20]:
ads.company

0                 \n\nCivil Service Fast Stream
1                                        \n\nBP
2                               \n\nMicro Focus
3                                 \nTangoe Inc.
4             \n\nForeign & Commonwealth Office
                        ...                    
14                                  \n\nFoodbuy
15    \n\nUK Government - National Crime Agency
16               \n\nOpus Recruitment Solutions
17              \n\nClearanswer Call Centre LTD
18                     \nGP Health Partners Ltd
Name: company, Length: 1895, dtype: object